<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Building_GNNmodel_with_PyG(First_Notbebook).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch-geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.1-py3-none-any.whl size=910459 sha256=8fa406ed70f90fdc433e1610fe5dd4c15c8f4f3b28fd117ba5bbd892f7369994
  Stored in directory: /root/.cache/pip/wheels/ac/dc/30/e2874821ff308ee67dcd7a66dbde912411e19e35a1addda028
Successfully built torch-geometric


# Importing Data and Libraries

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cBio = pd.read_csv('/content/drive/MyDrive/gsoc data/cBio.csv')

In [ ]:
cBio = cBio.set_index('Sample Identifier')
cBio.index.name = None
cBio.head(2)

,Overall Survival (Months),NUDT18,SHC3,NCRNA00112,FAM49B,FMO6P,BAT2L2,SIRT1,MSLNL,ATG4A,...,SCARNA6,SNORD77,TLR10,GKN2,PCDHGB4,ZMAT5,AXIN2,LONRF1,KLHL10,C12orf36
TCGA-OR-A5J1-01,44.547457,85.0144,97.5024,0.0,671.950,0.0,2428.43,462.056,0.0,385.687,...,0.0000,0.0,12.0077,0.0,17.2911,913.535,547.55,363.112,1.9212,0.0
TCGA-OR-A5J2-01,55.133642,165.8840,92.6050,0.0,812.911,0.0,5097.30,669.977,0.0,547.980,...,0.4026,0.0,0.8053,0.0,268.9890,303.978,1793.32,592.672,0.0000,0.0


In [ ]:
#defining features and labels

y = np.array(cBio['Overall Survival (Months)'], dtype=float)
X = cBio.drop('Overall Survival (Months)', axis=1)

In [ ]:
path = "/content/drive/MyDrive/gsoc data/PathwayCommons12.reactome.hgnc.sif.gz"  


#specify compression type because the file is compressed
df = pd.read_csv(path, sep="\t", compression="gzip", header=None, names=["Source", "InteractionType", "Target"])

In [ ]:
df = df.drop(columns='InteractionType')

In [ ]:
df.head()

,Source,Target
0,A1CF,APOBEC1
1,A1CF,APOBEC2
2,A1CF,APOBEC3A
3,A1CF,APOBEC3B
4,A1CF,APOBEC3C


# Selecting Common Nodes From Both Data

In [ ]:
all_nodes = sorted(list(set(df['Source']) | set(df['Target'])))
len(all_nodes)

12324

In [ ]:
len(set(all_nodes) & set(X.columns))

9614

In [ ]:
used_nodes = sorted(list(set(X.columns) & set(all_nodes)))
len(used_nodes)

9614

In [ ]:
X = X.loc[:, used_nodes]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAC,AADAT,AAK1,AANAT,AARS,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,0.0000,121.0370,868.876,1.4409,3520.65,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,1.2079,31.0025,1627.030,0.0000,5470.94,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


In [ ]:
# Extract the values from the 'Source' column of the DataFrame
source_values = df['Source'].values

# Create a boolean mask indicating whether each element in 'source_values' is present in 'used_nodes'
mask = np.isin(source_values, used_nodes)

df = df.loc[mask]

# Update 'source_values' with the values from the 'Target' column of the filtered DataFrame
source_values = df['Target'].values

# Create a new boolean mask based on the updated 'source_values' array
mask = np.isin(source_values, used_nodes)


df = df.loc[mask]

# Reset the index of the filtered DataFrame, dropping the old index
df = df.reset_index(drop=True)

In [ ]:
len(set(df['Source']) | set(df['Target']))

9288

In [ ]:

# Select the columns from DataFrame 'X' that correspond to unique values 
columns_to_select = sorted(set(df['Source']).union(set(df['Target'])))
X = X[columns_to_select]
X.head(2)

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-OR-A5J1-01,0.0,10373.70,0.0,2225.26,1266.090,121.0370,868.876,1.4409,3520.65,299.712,...,0.0,306.9160,0.0,2180.12,256.004,0.0000,253.602,183.958,146.974,596.062
TCGA-OR-A5J2-01,0.0,9844.91,0.0,1509.46,981.613,31.0025,1627.030,0.0000,5470.94,408.666,...,0.0,99.4255,0.0,2553.48,104.684,0.4026,270.165,264.125,438.464,801.637


# Creating Edge Index from Pathway Commons

In [ ]:
source_nodes = df['Source'].tolist()
target_nodes = df['Target'].tolist()


# Create a dictionary to map each unique node to a unique index
node_to_index = {node: index for index, node in enumerate(X.columns)}

# Map the source and target nodes to their corresponding indices
source_indices = [node_to_index[node] for node in source_nodes]
target_indices = [node_to_index[node] for node in target_nodes]

In [ ]:
# Convert the source and target indices to a PyTorch tensor
edge_index = torch.tensor([source_indices, target_indices], dtype=torch.long)

In [ ]:
edge_index

tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])

In [ ]:
edge_index.shape

torch.Size([2, 271771])

# Splitting the Data into Train and Test Splits

In [ ]:
# Set the fixed index for splitting
split_index = int(0.8 * X.shape[0])

# Split the data based on the fixed index
X_train = X[:split_index]
X_test = X[split_index:]
y_train = y[:split_index]
y_test = y[split_index:]

In [ ]:
X_train.shape, X_test.shape

((62, 9288), (16, 9288))

In [ ]:
X_train = X_train.values
X_test = X_test.values

In [ ]:
X_train[0]

array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
         596.062])

In [ ]:
X_test[1]

array([   0.    , 1728.03  ,    0.    , ...,  106.475 ,   39.4796,
        261.403 ])

In [ ]:
y_train

array([ 44.54745701,  55.13364237,  68.74445212,  11.9998685 ,
        88.86477956,  16.1094125 ,  19.03540783,  44.44882796,
        30.31199658,  18.11486997,  57.53361607,  99.8783575 ,
        69.20472104,  66.24584936,  17.78610645,  76.63477661,
        16.1094125 ,  49.215899  ,  67.59377979,  18.47650985,
        39.25436434,  15.25462735,  36.2626163 , 121.2479863 ,
        12.59164283,  29.8188513 ,  66.50886018,  72.39372719,
        31.23253444,  18.14774633,  27.02436138,  33.82976625,
       108.1303219 ,  32.67909393, 113.9165598 ,  35.57221291,
        16.37242332,  49.0843936 ,  24.62438768,  11.3094651 ,
        46.48716179,  95.17703916, 153.6311931 , 127.4944932 ,
        68.25130684,  44.84334418,  12.85465365,   4.10954401,
       152.1517572 ,  31.79143242,  43.29815564,  28.30653911,
        29.09557156,  28.63530263,  23.63809712,  39.58312786,
         5.22733997,  39.35299339,  21.76414505,  52.24052339,
        78.41009962,  36.328369  ])

# Generating patient-specific graphs

In [ ]:
num_patients_train = X_train.shape[0]
num_patients_test = X_test.shape[0]

# Create patient-specific graphs for the training set
graphs_train = []
for i in range(num_patients_train):
    node_features = X_train[i]  # Node features for the i-th patient
    target = y_train[i]  # Target label for the i-th patient
    graph_train = (node_features, edge_index, target)
    graphs_train.append(graph_train)

# Create patient-specific graphs for the test set
graphs_test = []
for i in range(num_patients_test):
    node_features = X_test[i]  # Node features for the i-th patient
    target = y_test[i]  # Target label for the i-th patient
    graph_test = (node_features, edge_index, target)
    graphs_test.append(graph_test)

In [ ]:
# Check the number of patient-specific graphs
print(len(graphs_train))  # Should be 62
print(len(graphs_test))  # Should be 16

62
16


In [ ]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 0 # Index of the patient
node_features, edge_index, y = graphs_train[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[    0.    10373.7       0.    ...   183.958   146.974   596.062]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
44.54745701


In [ ]:
# Access the patient-specific graph for a specific patient in the training set
patient_index = 15  # Index of the patient
node_features, edge_index, y = graphs_test[patient_index]
print(node_features)  # Node features for the specific patient
print(edge_index)  # Edge index for the specific patient
print(y)

[1.20290e+00 1.17995e+04 0.00000e+00 ... 9.13745e+02 1.31836e+03
 6.15878e+02]
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
42.50912319


# Converting List of Graphs to Data Objects

In [ ]:
# Convert graphs_train to a list of Data objects
data_train = [Data(x=torch.tensor(graph[0].reshape(len(graphs_train[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_train]

# Convert graphs_test to a list of Data objects
data_test = [Data(x=torch.tensor(graph[0].reshape(len(graphs_test[0][0]), 1)), edge_index=graph[1], y=torch.tensor(graph[2])) for graph in graphs_test]

In [ ]:
graphs_train[0][0]

array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
         596.062])

In [ ]:
graphs_train[0]

(array([    0.   , 10373.7  ,     0.   , ...,   183.958,   146.974,
          596.062]),
 tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
         [ 451,  452,  453,  ..., 3323, 3340, 3341]]),
 44.54745701)

In [ ]:
data_test

[Data(x=[9288, 1], edge_index=[2, 271771], y=90.08120459),
 Data(x=[9288, 1], edge_index=[2, 271771], y=53.02955584),
 Data(x=[9288, 1], edge_index=[2, 271771], y=61.08426209),
 Data(x=[9288, 1], edge_index=[2, 271771], y=76.99641648),
 Data(x=[9288, 1], edge_index=[2, 271771], y=79.06762666),
 Data(x=[9288, 1], edge_index=[2, 271771], y=61.05138574),
 Data(x=[9288, 1], edge_index=[2, 271771], y=28.04352829),
 Data(x=[9288, 1], edge_index=[2, 271771], y=36.03248184),
 Data(x=[9288, 1], edge_index=[2, 271771], y=18.04911727),
 Data(x=[9288, 1], edge_index=[2, 271771], y=38.49820824),
 Data(x=[9288, 1], edge_index=[2, 271771], y=6.805404872),
 Data(x=[9288, 1], edge_index=[2, 271771], y=24.85452214),
 Data(x=[9288, 1], edge_index=[2, 271771], y=119.1110234),
 Data(x=[9288, 1], edge_index=[2, 271771], y=20.25183286),
 Data(x=[9288, 1], edge_index=[2, 271771], y=39.4844988),
 Data(x=[9288, 1], edge_index=[2, 271771], y=42.50912319)]

In [ ]:
# Access the attributes of a specific data object in the training set
sample = data_train[0]  # Get the first data object
print(sample)  # Print the data object

# Access the node features, edge indices, and target label
node_features = sample.x
edge_index = sample.edge_index
target = sample.y

print(node_features)  # Print the node features
print(edge_index)  # Print the edge indices
print(target)  # Print the target label

Data(x=[9288, 1], edge_index=[2, 271771], y=44.54745701)
tensor([[    0.0000],
        [10373.7000],
        [    0.0000],
        ...,
        [  183.9580],
        [  146.9740],
        [  596.0620]], dtype=torch.float64)
tensor([[   0,    0,    0,  ..., 9287, 9287, 9287],
        [ 451,  452,  453,  ..., 3323, 3340, 3341]])
tensor(44.5475, dtype=torch.float64)


# Creating Train and Test Batches

In [ ]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(data_train, batch_size=32, shuffle=True)
test_loader = DataLoader(data_test, batch_size=8, shuffle=False)

for step, data in enumerate(train_loader):
    print('Training Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Training Batches: 
Step 1:
Number of graphs in the current batch: 32
DataBatch(x=[297216, 1], edge_index=[2, 8696672], y=[32], batch=[297216], ptr=[33])

Training Batches: 
Step 2:
Number of graphs in the current batch: 30
DataBatch(x=[278640, 1], edge_index=[2, 8153130], y=[30], batch=[278640], ptr=[31])



In [ ]:
for step, data in enumerate(test_loader):
    print('Test Batches: ')
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Test Batches: 
Step 1:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])

Test Batches: 
Step 2:
Number of graphs in the current batch: 8
DataBatch(x=[74304, 1], edge_index=[2, 2174168], y=[8], batch=[74304], ptr=[9])



# Model Building and Evaluation

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 1)  # Regression output with 1 dimension

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final regression layer
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)

        return x.squeeze()  # Remove the extra dimension

In [41]:
model = GCN(hidden_channels=64, num_node_features=1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.double()  # Convert the model's parameters to Double type

num_epochs = 101  # Specify the number of epochs

for epoch in range(num_epochs):
    model.train()  # Set the model to train mode
    total_loss = 0

    for step, data in enumerate(train_loader):
        optimizer.zero_grad()

        out = model(data.x.double(), data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1).double())
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / (step + 1)
    
    # Evaluate on the test set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0
        for data in test_loader:
            out = model(data.x.double(), data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1).double())
            test_loss += loss.item()

        average_test_loss = test_loss / len(data_test)
        #print(f"Epoch: {epoch+1:03d}, Test Loss: {average_test_loss:.4f}")
        print(f'Epoch: {epoch:03d}, Train loss: {average_loss:.4f}, Test Loss: {average_test_loss:.4f}')

Epoch: 000, Train loss: 129410.2112, Test Loss: 791.3781
Epoch: 001, Train loss: 23701.5885, Test Loss: 5243.8372
Epoch: 002, Train loss: 34177.5206, Test Loss: 600.6721
Epoch: 003, Train loss: 3650.1808, Test Loss: 246.1246
Epoch: 004, Train loss: 2968.0425, Test Loss: 347.6179
Epoch: 005, Train loss: 3282.8482, Test Loss: 418.1660
Epoch: 006, Train loss: 3647.8534, Test Loss: 421.4515
Epoch: 007, Train loss: 3548.1207, Test Loss: 409.9760
Epoch: 008, Train loss: 3553.1150, Test Loss: 397.4286
Epoch: 009, Train loss: 3395.7473, Test Loss: 374.8122
Epoch: 010, Train loss: 3209.0978, Test Loss: 345.3970
Epoch: 011, Train loss: 2973.3637, Test Loss: 309.0914
Epoch: 012, Train loss: 2734.1807, Test Loss: 264.8467
Epoch: 013, Train loss: 2345.7370, Test Loss: 214.0014
Epoch: 014, Train loss: 1973.3042, Test Loss: 161.6546
Epoch: 015, Train loss: 1732.9938, Test Loss: 118.0704
Epoch: 016, Train loss: 1290.9724, Test Loss: 94.3744
Epoch: 017, Train loss: 1297.6568, Test Loss: 93.3776
Epoch: 

In [42]:
model.eval()  # Set the model to evaluation mode

predictions = []  # List to store the predicted outputs

with torch.no_grad():
    for data in test_loader:
        out = model(data.x.double(), data.edge_index, data.batch)
        predictions.append(out.detach().numpy())  # Convert the predictions to NumPy array

# Concatenate the predictions from multiple batches
predictions = np.concatenate(predictions)

# Print the predictions
print(predictions)

[53.01817626 40.05074882 49.00959863 44.54161161 35.76659514 44.95573184
 46.62116467 38.56213906 39.27010396 39.03080886 39.52077349 47.93688063
 51.97068159 44.27898666 44.72378354 45.02683638]


In [45]:
from sklearn.metrics import r2_score

# Convert the test data batches to a list of Data objects
test_data_batches = [
    Data(x=batch.x, edge_index=batch.edge_index, y=batch.y) for batch in test_data_batches
]

# Convert the predictions to PyTorch tensors
predictions = torch.tensor(predictions)

# Convert the ground truth labels of the test data to a PyTorch tensor
y_true = torch.cat([batch.y for batch in test_data_batches])

# Calculate the mean squared error (MSE) loss using PyTorch's function
mse_loss = torch.nn.functional.mse_loss(predictions.view(-1), y_true.view(-1))

# Calculate the R-squared (coefficient of determination) using scikit-learn's function
r2 = r2_score(y_true.view(-1).numpy(), predictions.view(-1).numpy())

print(f"Mean Squared Error (MSE) Loss: {mse_loss:.4f}")
print(f"R-squared (Accuracy): {r2:.4f}")


Mean Squared Error (MSE) Loss: 774.9644
R-squared (Accuracy): 0.0876


<ipython-input-45-d38790cb8a51>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predictions = torch.tensor(predictions)
